# YOLOv8n Traffic Light Detection Training on Kaggle

This notebook trains a YOLOv8n model for traffic light detection using Kaggle's free GPU.

## Prerequisites
- Upload your dataset to Kaggle Datasets
- Enable GPU accelerator (Settings → Accelerator → GPU P100)
- Make sure your dataset follows YOLO format with `dataset.yaml`

## 1. Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install required packages
!pip install -q ultralytics
!pip install -q roboflow  # Optional: if using Roboflow datasets

In [ ]:
# Import libraries
import os
import yaml
import shutil
from pathlib import Path
from ultralytics import YOLO
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Dataset Setup

### Option A: Using Kaggle Dataset
If you uploaded your dataset to Kaggle, add it as a data source in the notebook settings.

In [ ]:
# Set dataset path - Update this to match your Kaggle dataset path
# Example: /kaggle/input/your-dataset-name/
DATASET_PATH = '/kaggle/input/traffic-light-dataset/'  # Change this!

# Verify dataset structure
print("Dataset contents:")
!ls -la {DATASET_PATH}

### Option B: Download from Roboflow or URL

In [ ]:
# Uncomment if downloading from Roboflow
# from roboflow import Roboflow
# rf = Roboflow(api_key="YOUR_API_KEY")
# project = rf.workspace("YOUR_WORKSPACE").project("YOUR_PROJECT")
# dataset = project.version(1).download("yolov8")
# DATASET_PATH = dataset.location

## 3. Verify Dataset Configuration

In [ ]:
# Find and read dataset.yaml
yaml_path = os.path.join(DATASET_PATH, 'dataset.yaml')
# Alternative common names
if not os.path.exists(yaml_path):
    yaml_path = os.path.join(DATASET_PATH, 'data.yaml')

print(f"Using config: {yaml_path}")

# Read and display dataset config
with open(yaml_path, 'r') as f:
    dataset_config = yaml.safe_load(f)
    print("\nDataset Configuration:")
    print(yaml.dump(dataset_config, default_flow_style=False))

In [ ]:
# Update paths in dataset.yaml if needed
# This is important if the yaml has relative paths
with open(yaml_path, 'r') as f:
    config = yaml.safe_load(f)

# Update path to absolute path
config['path'] = DATASET_PATH

# Save updated config
updated_yaml_path = '/kaggle/working/dataset.yaml'
with open(updated_yaml_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print(f"Updated config saved to: {updated_yaml_path}")
yaml_path = updated_yaml_path

In [ ]:
# Verify dataset structure
train_images = os.path.join(DATASET_PATH, config.get('train', 'train/images'))
val_images = os.path.join(DATASET_PATH, config.get('val', 'val/images'))

print(f"\nTrain images path: {train_images}")
print(f"Exists: {os.path.exists(train_images)}")
if os.path.exists(train_images):
    train_count = len([f for f in os.listdir(train_images) if f.endswith(('.jpg', '.jpeg', '.png'))])
    print(f"Train images count: {train_count}")

print(f"\nVal images path: {val_images}")
print(f"Exists: {os.path.exists(val_images)}")
if os.path.exists(val_images):
    val_count = len([f for f in os.listdir(val_images) if f.endswith(('.jpg', '.jpeg', '.png'))])
    print(f"Val images count: {val_count}")

print(f"\nNumber of classes: {config.get('nc', 'Not specified')}")
print(f"Class names: {config.get('names', 'Not specified')}")

## 4. Initialize YOLOv8n Model

In [ ]:
# Load YOLOv8n model
# Use pretrained COCO weights for transfer learning
model = YOLO('yolov8n.pt')

# Or start from scratch (not recommended)
# model = YOLO('yolov8n.yaml')

print("Model loaded successfully!")
print(f"Model type: {type(model)}")

## 5. Training Configuration

In [ ]:
# Training hyperparameters
EPOCHS = 100  # Adjust based on your needs
IMG_SIZE = 640  # Standard YOLO input size
BATCH_SIZE = 16  # Adjust based on GPU memory (P100 can handle 16-32)
PATIENCE = 20  # Early stopping patience
PROJECT_NAME = 'traffic_light_detection'
EXPERIMENT_NAME = 'yolov8n_run1'

print(f"Training Configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Image Size: {IMG_SIZE}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Patience: {PATIENCE}")
print(f"  Project: {PROJECT_NAME}")
print(f"  Experiment: {EXPERIMENT_NAME}")

## 6. Train the Model

In [ ]:
# Train the model
results = model.train(
    data=yaml_path,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    patience=PATIENCE,
    project=PROJECT_NAME,
    name=EXPERIMENT_NAME,
    device=0,  # Use GPU 0
    
    # Optimization settings
    optimizer='AdamW',  # or 'SGD', 'Adam'
    lr0=0.01,  # Initial learning rate
    lrf=0.01,  # Final learning rate (lr0 * lrf)
    momentum=0.937,  # SGD momentum/Adam beta1
    weight_decay=0.0005,  # Optimizer weight decay
    
    # Augmentation settings
    hsv_h=0.015,  # HSV-Hue augmentation
    hsv_s=0.7,  # HSV-Saturation augmentation
    hsv_v=0.4,  # HSV-Value augmentation
    degrees=0.0,  # Rotation (+/- deg)
    translate=0.1,  # Translation (+/- fraction)
    scale=0.5,  # Scaling (+/- gain)
    shear=0.0,  # Shear (+/- deg)
    perspective=0.0,  # Perspective (+/- fraction)
    flipud=0.0,  # Flip up-down (probability)
    fliplr=0.5,  # Flip left-right (probability)
    mosaic=1.0,  # Mosaic augmentation (probability)
    mixup=0.0,  # MixUp augmentation (probability)
    
    # Other settings
    save=True,  # Save checkpoints
    save_period=-1,  # Save checkpoint every x epochs (-1 = disabled)
    cache=False,  # Cache images for faster training (use True if enough RAM)
    workers=8,  # Number of worker threads
    pretrained=True,  # Use pretrained weights
    verbose=True,  # Verbose output
    seed=0,  # Random seed for reproducibility
    deterministic=True,  # Deterministic mode
    
    # Validation settings
    val=True,  # Validate during training
    plots=True,  # Save plots
)

print("\n" + "="*50)
print("Training completed!")
print("="*50)

## 7. Evaluate the Model

In [ ]:
# Load the best model
best_model_path = f'{PROJECT_NAME}/{EXPERIMENT_NAME}/weights/best.pt'
best_model = YOLO(best_model_path)

print(f"Loaded best model from: {best_model_path}")

In [ ]:
# Validate on validation set
metrics = best_model.val(
    data=yaml_path,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    conf=0.25,  # Confidence threshold
    iou=0.6,  # IoU threshold for NMS
    device=0,
    plots=True,
)

print("\n" + "="*50)
print("Validation Metrics:")
print("="*50)
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

In [ ]:
# Display per-class metrics
print("\nPer-class metrics:")
print("-" * 50)
for i, class_name in enumerate(config['names']):
    if isinstance(config['names'], dict):
        class_name = config['names'][i]
    print(f"{class_name}:")
    print(f"  AP50: {metrics.box.ap50[i]:.4f}")
    print(f"  AP: {metrics.box.ap[i]:.4f}")

## 8. Visualize Results

In [ ]:
# Display training plots
from IPython.display import Image, display
import glob

results_path = f'{PROJECT_NAME}/{EXPERIMENT_NAME}'

# Show confusion matrix
confusion_matrix_path = f'{results_path}/confusion_matrix.png'
if os.path.exists(confusion_matrix_path):
    print("Confusion Matrix:")
    display(Image(filename=confusion_matrix_path))

# Show results
results_img_path = f'{results_path}/results.png'
if os.path.exists(results_img_path):
    print("\nTraining Results:")
    display(Image(filename=results_img_path))

# Show PR curve
pr_curve_path = f'{results_path}/PR_curve.png'
if os.path.exists(pr_curve_path):
    print("\nPrecision-Recall Curve:")
    display(Image(filename=pr_curve_path))

# Show F1 curve
f1_curve_path = f'{results_path}/F1_curve.png'
if os.path.exists(f1_curve_path):
    print("\nF1 Curve:")
    display(Image(filename=f1_curve_path))

In [ ]:
# Show validation batch predictions
val_batches = glob.glob(f'{results_path}/val_batch*.jpg')
if val_batches:
    print("\nValidation Batch Predictions:")
    for batch_img in val_batches[:3]:  # Show first 3 batches
        print(f"\n{os.path.basename(batch_img)}:")
        display(Image(filename=batch_img, width=800))

## 9. Test on Sample Images

In [ ]:
# Get some test images
test_images_path = os.path.join(DATASET_PATH, config.get('test', 'test/images'))
if not os.path.exists(test_images_path):
    test_images_path = val_images  # Use val images if test not available

test_images = [os.path.join(test_images_path, f) for f in os.listdir(test_images_path) 
               if f.endswith(('.jpg', '.jpeg', '.png'))][:5]  # Get first 5 images

print(f"Testing on {len(test_images)} sample images...")

In [ ]:
# Run inference
predictions = best_model.predict(
    source=test_images,
    imgsz=IMG_SIZE,
    conf=0.25,
    iou=0.45,
    device=0,
    save=True,
    project=f'{PROJECT_NAME}/predictions',
    name='test_samples',
)

print(f"Predictions saved to: {PROJECT_NAME}/predictions/test_samples/")

In [ ]:
# Display predictions
pred_images = glob.glob(f'{PROJECT_NAME}/predictions/test_samples/*.jpg')
print(f"\nSample Predictions ({len(pred_images)} images):")
for pred_img in pred_images:
    print(f"\n{os.path.basename(pred_img)}:")
    display(Image(filename=pred_img, width=600))

## 10. Export Model

In [ ]:
# Export to different formats
# PyTorch format (already saved)
print(f"PyTorch model: {best_model_path}")

# Export to ONNX (for deployment)
onnx_path = best_model.export(format='onnx', imgsz=IMG_SIZE)
print(f"ONNX model exported to: {onnx_path}")

# Uncomment to export to other formats
# torchscript_path = best_model.export(format='torchscript')
# tflite_path = best_model.export(format='tflite')
# edgetpu_path = best_model.export(format='edgetpu')

## 11. Save Results and Download

In [ ]:
# Create a summary file
summary_path = f'{PROJECT_NAME}/{EXPERIMENT_NAME}/training_summary.txt'
with open(summary_path, 'w') as f:
    f.write("YOLOv8n Traffic Light Detection - Training Summary\n")
    f.write("="*60 + "\n\n")
    
    f.write("Training Configuration:\n")
    f.write(f"  Model: YOLOv8n\n")
    f.write(f"  Epochs: {EPOCHS}\n")
    f.write(f"  Image Size: {IMG_SIZE}\n")
    f.write(f"  Batch Size: {BATCH_SIZE}\n")
    f.write(f"  Dataset: {DATASET_PATH}\n")
    f.write(f"  Classes: {config.get('nc')}\n")
    f.write(f"  Class Names: {config.get('names')}\n\n")
    
    f.write("Validation Metrics:\n")
    f.write(f"  mAP50: {metrics.box.map50:.4f}\n")
    f.write(f"  mAP50-95: {metrics.box.map:.4f}\n")
    f.write(f"  Precision: {metrics.box.mp:.4f}\n")
    f.write(f"  Recall: {metrics.box.mr:.4f}\n\n")
    
    f.write("Per-class AP50:\n")
    for i, class_name in enumerate(config['names']):
        if isinstance(config['names'], dict):
            class_name = config['names'][i]
        f.write(f"  {class_name}: {metrics.box.ap50[i]:.4f}\n")
    
    f.write("\n" + "="*60 + "\n")
    f.write(f"Best model saved to: {best_model_path}\n")
    f.write(f"ONNX model saved to: {onnx_path}\n")

print(f"Training summary saved to: {summary_path}")
print("\nTraining summary:")
with open(summary_path, 'r') as f:
    print(f.read())

In [ ]:
# List all important files
print("\nImportant files to download:")
print("-" * 60)
important_files = [
    f'{PROJECT_NAME}/{EXPERIMENT_NAME}/weights/best.pt',
    f'{PROJECT_NAME}/{EXPERIMENT_NAME}/weights/last.pt',
    f'{PROJECT_NAME}/{EXPERIMENT_NAME}/results.csv',
    f'{PROJECT_NAME}/{EXPERIMENT_NAME}/results.png',
    f'{PROJECT_NAME}/{EXPERIMENT_NAME}/confusion_matrix.png',
    summary_path,
]

for file_path in important_files:
    if os.path.exists(file_path):
        size = os.path.getsize(file_path) / (1024 * 1024)  # MB
        print(f"✓ {file_path} ({size:.2f} MB)")
    else:
        print(f"✗ {file_path} (not found)")

In [ ]:
# Optional: Create a zip file with all results
import zipfile

zip_filename = f'{PROJECT_NAME}_{EXPERIMENT_NAME}_results.zip'
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Add weights
    zipf.write(f'{PROJECT_NAME}/{EXPERIMENT_NAME}/weights/best.pt', 'weights/best.pt')
    zipf.write(f'{PROJECT_NAME}/{EXPERIMENT_NAME}/weights/last.pt', 'weights/last.pt')
    
    # Add results
    for file in glob.glob(f'{PROJECT_NAME}/{EXPERIMENT_NAME}/*.png'):
        zipf.write(file, f'results/{os.path.basename(file)}')
    
    # Add CSV results
    if os.path.exists(f'{PROJECT_NAME}/{EXPERIMENT_NAME}/results.csv'):
        zipf.write(f'{PROJECT_NAME}/{EXPERIMENT_NAME}/results.csv', 'results.csv')
    
    # Add summary
    zipf.write(summary_path, 'training_summary.txt')

print(f"\nAll results packaged in: {zip_filename}")
print(f"Size: {os.path.getsize(zip_filename) / (1024 * 1024):.2f} MB")
print("\nYou can download this file from the Kaggle output panel.")

## 12. Model Information

In [ ]:
# Display model information
print("Model Architecture Summary:")
print("="*60)
best_model.info(verbose=True)

# Count parameters
total_params = sum(p.numel() for p in best_model.model.parameters())
trainable_params = sum(p.numel() for p in best_model.model.parameters() if p.requires_grad)

print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model size: {os.path.getsize(best_model_path) / (1024 * 1024):.2f} MB")

## Notes

### Tips for Better Results:
1. **Increase epochs**: Try 150-300 epochs for better convergence
2. **Adjust batch size**: Larger batch size (32-64) if GPU memory allows
3. **Enable caching**: Set `cache=True` if you have enough RAM (speeds up training)
4. **Data augmentation**: Adjust augmentation parameters based on your dataset
5. **Learning rate**: Experiment with different learning rates (0.001 - 0.01)
6. **Image size**: Try 1280 for better accuracy (slower training)

### Common Issues:
- **Out of memory**: Reduce batch size or image size
- **Poor performance**: Check dataset quality, increase epochs, adjust augmentation
- **Overfitting**: Increase augmentation, add more training data
- **Slow convergence**: Adjust learning rate, check dataset balance

### Next Steps:
1. Download the trained model (`best.pt`)
2. Test on real-world images
3. Fine-tune hyperparameters if needed
4. Deploy to your application

### Resources:
- [Ultralytics YOLOv8 Docs](https://docs.ultralytics.com/)
- [YOLOv8 Training Tips](https://docs.ultralytics.com/modes/train/)
- [Hyperparameter Tuning](https://docs.ultralytics.com/guides/hyperparameter-tuning/)